#### Antiga forma de criar requests

Os requests são tuplas de ```(app, alice, bob, prioridade)```. Trocado por um objeto Request porque facilitava a atualização das chaves obtidas. Caminho antigo: ```components/qkd/app.py``` e ```components/controller.py```.

In [ ]:
# Simulação de QKD

def generate_qkd_request(rede, num_requests, avaliable_apps, diff_nodes=5):
        """
        Gera uma lista de requisições aleatórias de QKD.

        Args:
            num_requests (int): Número de requisições.
            diff_nodes (int): Número entre os nós. Defauts to 5.

        Returns:
            requests (list): Lista com requisições.
        """
        requests = []
        
        for i in range(num_requests):
            alice, bob = rede.random_alice_bob(diff_nodes)
            priority = random.randint(1, 5)
            app = random.choice(avaliable_apps)
            requests.append((alice, bob, app, priority))
        
        return requests
    
def run_simulations(rede, controlador, n_simulacoes, n_requests, avaliable_apps, routes_calculation_type='shortest'):
    """
    Roda as simulações para os protocolos BB84, E91 e B92.

    Args:
        rede (Network): Rede.
        controlador (Controller): Controlador.
        n_simulacoes (int): Número de simulações.
        n_requests (int): Número de requisições.
        routes_calculation_type (str): Tipo de roteamento. Defaults to 'shortest'.
    
    Returns:
        taxas_sucesso_chaves_geral (list): Lista com as taxas de sucesso das chaves para cada simulação.
        vazao (list): Lista com a vazão para cada simulação.
    """
    
    taxas_sucesso_chaves_geral = []
    vazao = []

    for simulacao in range(n_simulacoes):
        print("Simulação: ", simulacao)
        taxas_sucesso_chaves_e91 = []
        taxas_sucesso_chaves_bb84 = []
        taxas_sucesso_chaves_b92 = []
        
        requests = generate_qkd_request(rede, n_requests, avaliable_apps)
        print("Requests: ", requests)
        resultados_simulacao = controlador.send_requests(requests, routes_calculation_type)

        for indice_execucao in resultados_simulacao:
            resultado_individual_simulacao = resultados_simulacao[indice_execucao]
            sucesso_chave = resultado_individual_simulacao['key sucess']

            if resultado_individual_simulacao['app'] == 'BB84':
                taxas_sucesso_chaves_bb84.append(sucesso_chave)
                
            elif resultado_individual_simulacao['app'] == 'E91':
                taxas_sucesso_chaves_e91.append(sucesso_chave)
                
            elif resultado_individual_simulacao['app'] == 'B92':
                taxas_sucesso_chaves_b92.append(sucesso_chave)

        # Salvando o sucesso nas chaves geral da simulação
        lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
        taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
        
        # Calculando a vazão
        n_execucoes = len(resultados_simulacao)
        vazao.append(n_requests / n_execucoes)

    return taxas_sucesso_chaves_geral, vazao

In [ ]:
    
def allocate_routes(self, request_list, routes_calculation_type='shortest'):
    """
    Aloca as rotas para os pedidos e atualiza a lista de requisições.
    
    Args:
        request_list (list): Lista de requisições -> [alice, bob, app].
    
    Returns:
        info (list): Lista de dicionários -> [{'Priority': priority, 'Route': route, 'App': app}].
    """

    # Informações das requisições
    info = []
    # Todos os links já utilizados
    all_used_links = set()
    # Links utilizados apenas por BB84 e B92
    bb84_b92_used_links = set()
    # Contador de rotas E91 (para que não haja muitos E91 com links compartilhados)
    e91_count = 0
    # Ordenando as requisições por prioridade
    sorted_request_list = sorted(request_list, key=lambda x: x[3], reverse=True)
                    
    for alice, bob, app, priority in sorted_request_list:
        
        # Calcula as rotas de menor custo
        if routes_calculation_type == 'shortest':
            routes = self.calculate_shortest_routes(alice, bob)
        elif routes_calculation_type == 'kshortest': 
            routes = self.calculate_k_shortest_routes(alice, bob)
        elif routes_calculation_type == 'all':
            routes = self.calculate_all_routes(alice, bob)
        elif routes_calculation_type == 'klength':
            routes = self.calculate_routes_of_k_length(alice, bob, 5)
        
        #print(f'Rotas: {routes}')
        # Itera sobre as rotas
        for route in routes:
            #print(f'Rota Avaliada: {route}')
            # Lista de pares de elementos adjacentes da lista route (canais)
            route_links = [(route[i], route[i + 1]) for i in range(len(route) - 1)]

            # Checa se nenhum link dessa rota já foi usado em uma rota anterior
            if not any(link in all_used_links for link in route_links):
                    # Se a app for BB84 ou B92, adiciona os links usados no conjunto de links usados apenas por essas apps
                    if app == 'BB84' or app == 'B92':
                        bb84_b92_used_links.update(route_links)
                    
                    # Adicionando as informações de rota, app e prioridade
                    info.append({'Priority': priority, 'Route': route, 'App': app})
                    
                    # Adiciona os links usados no conjunto de links usados
                    all_used_links.update(route_links)
                    
                    # Remove a requisição da lista de requisições
                    request_list.remove((alice, bob, app, priority))
                    
                    break
                
            elif app == 'E91':
                # Se nenhum dos links desta rota está nos links utilizados pelos outros tipos de protocolo, os links que estão sendo utilizados são dos outros E91
                if not any(e91_link in bb84_b92_used_links for e91_link in route_links):
                    # Para que não haja tantos E91 com links compartilhados, só 3 rotas E91 que compartilham links serão alocadas no máximo
                    if e91_count < 3:
                        # Adicionando as informações de rota, app e prioridade
                        info.append({'Priority': priority, 'Route': route, 'App': app})
                        
                        # Adicionando o link ao conjunto de links usados
                        all_used_links.update(route_links)
                        
                        # Removendo a requisição da lista de requisições
                        request_list.remove((alice, bob, app, priority))
                        e91_count += 1
                    
                    break
    
    return info

def send_requests(self, request_list, routes_calculation_type='shortest'):
    """
    Envia as requisições para a rede que as executa a partir da lista.
    
    Args:
        request_list (list): Lista de requisições -> [alice, bob, app].
    """

    # Índice da execução
    exec_index = 1
    results = dict()
    
    # Enquanto houver requisições na lista de requisições
    while len(request_list) > 0:
        # print("Requisições: ", request_list, "\n")
        # Alocando as rotas para os pedidos possíveis de serem atendidos, ou seja, que não compartilham links
        requests_info = self.allocate_routes(request_list, routes_calculation_type)
                    
        #LOG
        print(f'{exec_index}ª execução.')
        #print(f'Rotas alocadas: {allocated_routes}')
        #print(f'Apps: {list_app}')
        
        for request in requests_info:
            # Informações da requisição
            alice, bob = request["Route"][0], request["Route"][-1]
            app = request["App"]
            route = request["Route"]
            
            # Executa a aplicação QKD
            if app == 'B92':
                exec_data = run_qkd_b92(self.network, route)
            elif app == 'BB84':
                exec_data = run_qkd_bb84(self.network, route)
            elif app == 'E91':
                exec_data = run_qkd_e91(self.network, route)
        
        # Coletando dados
        results[exec_index] = exec_data
        exec_index += 1
    
    return results
